Based on https://bambinos.github.io/bambi/notebooks/radon_example.html

In order to render the graphical model this notebook requires `graphviz` to be installed and available via `PATH`.

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

In [ ]:
az.style.use("arviz-darkgrid")
np.random.default_rng(8924)

In [ ]:
# Get radon data
path = "https://raw.githubusercontent.com/pymc-devs/pymc-examples/main/examples/data/srrs2.dat"
radon_df = pd.read_csv(path)

# Get city data
city_df = pd.read_csv(pm.get_data("cty.dat"))

In [ ]:
# Strip spaces from column names
radon_df.columns = radon_df.columns.map(str.strip)

# Filter to keep observations for "MN" state only
df = radon_df[radon_df.state == "MN"].copy()
city_mn_df = city_df[city_df.st == "MN"].copy()

# Compute fips
df["fips"] = 1_000 * df.stfips + df.cntyfips
city_mn_df["fips"] = 1_000 * city_mn_df.stfips + city_mn_df.ctfips

# Merge data
df = df.merge(city_mn_df[["fips", "Uppm"]], on="fips")
df = df.drop_duplicates(subset="idnum")

# Clean county names
df.county = df.county.map(str.strip)

# Compute log(radon + 0.1)
df["log_radon"] = np.log(df["activity"] + 0.1)

# Compute log of Uranium
df["log_u"] = np.log(df["Uppm"])

# Let's map floor. 0 -> Basement and 1 -> Floor
df["floor"] = df["floor"].map({0: "Basement", 1: "Floor"})

# Sort values by floor
df = df.sort_values(by="floor")

# Reset index
df = df.reset_index(drop=True)

In [ ]:
_, ax = plt.subplots()
sns.histplot(
    x="activity",
    alpha=0.2,
    stat="density",
    element="step",
    common_norm=False,
    data=df,
    ax=ax,
)
sns.kdeplot(x="activity", data=df, ax=ax, cut=0)
ax.set(title="Density of Radon", xlabel="Radon", ylabel="Density");

In [ ]:
_, ax = plt.subplots()
sns.histplot(
    x="log_radon",
    alpha=0.2,
    stat="density",
    element="step",
    common_norm=False,
    data=df,
    ax=ax,
)
sns.kdeplot(x="log_radon", data=df, ax=ax)
ax.set(
    title="Density of log(Radon + 0.1)", xlabel="$\log(Radon + 0.1)$", ylabel="Density"
);

In [ ]:
_, ax = plt.subplots()
sns.histplot(
    x="log_radon",
    hue="floor",
    alpha=0.2,
    stat="density",
    element="step",
    common_norm=False,
    data=df,
    ax=ax,
)
sns.kdeplot(x="log_radon", hue="floor", common_norm=False, data=df, ax=ax)
ax.set(title="Density of log(Radon + 0.1)", xlabel="$\log + 0.1$", ylabel="Density");

In [ ]:
n_counties = df["county"].unique().size
print(f"Number of counties: {n_counties}")

In [ ]:
log_radon_county_agg = df.groupby(["county", "floor"], as_index=False).agg(
    log_radon_mean=("log_radon", "mean"), n_obs=("log_radon", "count")
)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), layout="constrained")
sns.boxplot(x="floor", y="log_radon_mean", data=log_radon_county_agg, ax=ax[0])
ax[0].set(title="log(Radon + 0.1) Mean per County", ylabel="$\log + 0.1$")

sns.boxplot(x="floor", y="n_obs", data=log_radon_county_agg, ax=ax[1])
ax[1].set(
    title="Number of Observations", xlabel="floor", ylabel="Number of observations"
);

In [ ]:
assert df.query("county == 'YELLOW MEDICINE' and floor == 'Floor'").empty

In [ ]:
# A dictionary with the priors we pass to the model initialization
pooled_priors = {
    "floor": bmb.Prior("Normal", mu=0, sigma=10),
    "sigma": bmb.Prior("Exponential", lam=1),
}

pooled_model = bmb.Model("log_radon ~ 0 + floor", df, priors=pooled_priors)
pooled_model

In [ ]:
pooled_model.build()
pooled_model.graph()

In [ ]:
pooled_results = pooled_model.fit()

In [ ]:
az.plot_trace(data=pooled_results, compact=True, chain_prop={"ls": "-"})
plt.suptitle("Pooled Model Trace");

In [ ]:
pooled_summary = az.summary(data=pooled_results)
pooled_summary

In [ ]:
unpooled_priors = {
    "county:floor": bmb.Prior("Normal", mu=0, sigma=10),
    "sigma": bmb.Prior("Exponential", lam=1),
}

unpooled_model = bmb.Model("log_radon ~ 0 + county:floor", df, priors=unpooled_priors)
unpooled_model

In [ ]:
try:
    unpooled_results = unpooled_model.fit()
except EOFError:
    pass

Stopped here with the unpooled model because EOFError produced on apple silicon as [discussed here](https://github.com/bambinos/bambi/issues/700) without solution.

In [ ]:
# We can add the hyper-priors inside the prior dictionary parameter of the model constructor
partial_pooling_priors = {
    "Intercept": bmb.Prior("Normal", mu=0, sigma=10),
    "1|county": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("Exponential", lam=1)),
    "sigma": bmb.Prior("Exponential", lam=1),
}

partial_pooling_model = bmb.Model(
    formula="log_radon ~ 1 + (1|county)",
    data=df,
    priors=partial_pooling_priors,
    noncentered=False,
)
partial_pooling_model

In [ ]:
partial_pooling_results = partial_pooling_model.fit()

In [ ]:
partial_pooling_model.graph()

In [ ]:
az.plot_trace(data=partial_pooling_results, compact=True, chain_prop={"ls": "-"})
plt.suptitle("Partial Pooling Model Trace");

In [ ]:
partial_pooling_model.predict(partial_pooling_results, kind="pps")

# Stack chains and draws. pps stands for posterior predictive samples
pps = az.extract_dataset(partial_pooling_results, group="posterior_predictive")[
    "log_radon"
].values

pps_df = pd.DataFrame(data=pps).assign(county=df["county"])
y_pred = pps_df.groupby("county").mean().mean(axis=1)
y_sample = df.groupby("county")["log_radon"].mean()

fig, ax = plt.subplots(figsize=(8, 7))
sns.regplot(x=y_sample, y=y_pred, ax=ax)
ax.axline(xy1=(1, 1), slope=1, color="black", linestyle="--", label="diagonal")
ax.axhline(y=y_pred.mean(), color="C3", linestyle="--", label="predicted global mean")
ax.legend(loc="lower right")
ax.set(
    title="log(Radon + 0.1) Mean per County (Partial Pooling Model)",
    xlabel="observed (sample)",
    ylabel="prediction",
    xlim=(0.3, 2.7),
    ylim=(0.3, 2.7),
);

In [ ]:
varying_intercept_priors = {
    "floor": bmb.Prior("Normal", mu=0, sigma=10),
    "1|county": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("Exponential", lam=1)),
    "sigma": bmb.Prior("Exponential", lam=1),
}

varying_intercept_model = bmb.Model(
    formula="log_radon ~ 0 + floor + (1|county)",
    data=df,
    priors=varying_intercept_priors,
    noncentered=False,
)

varying_intercept_model

In [ ]:
varying_intercept_results = varying_intercept_model.fit()

In [ ]:
varying_intercept_model.graph()

In [ ]:
az.plot_trace(data=varying_intercept_results, compact=True, chain_prop={"ls": "-"})
plt.suptitle("Varying Intercepts Model Trace");

In [ ]:
varying_intercept_slope_priors = {
    "floor": bmb.Prior("Normal", mu=0, sigma=10),
    "floor|county": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("Exponential", lam=1)),
    "sigma": bmb.Prior("Exponential", lam=1),
}

varying_intercept_slope_model = bmb.Model(
    formula="log_radon ~ 0 + floor + (0 + floor|county)",
    data=df,
    priors=varying_intercept_slope_priors,
    noncentered=True,
)

varying_intercept_slope_model

In [ ]:
varying_intercept_slope_results = varying_intercept_slope_model.fit(
    draws=2000, tune=2000, target_accept=0.9
)

In [ ]:
varying_intercept_slope_model.graph()

In [ ]:
var_names = ["floor", "floor|county", "floor|county_sigma", "sigma"]
az.plot_trace(
    data=varying_intercept_slope_results,
    var_names=var_names,
    compact=True,
    chain_prop={"ls": "-"},
);

In [ ]:
covariate_priors = {
    "floor": bmb.Prior("Normal", mu=0, sigma=10),
    "log_u": bmb.Prior("Normal", mu=0, sigma=10),
    "floor|county": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("Exponential", lam=1)),
    "sigma": bmb.Prior("Exponential", lam=1),
}

covariate_model = bmb.Model(
    formula="log_radon ~ 0 + floor + log_u + (0 + floor|county)",
    data=df,
    priors=covariate_priors,
    noncentered=True,
)

covariate_model

In [ ]:
covariate_results = covariate_model.fit(
    draws=2000, tune=2000, target_accept=0.9, chains=2
)

In [ ]:
covariate_model.graph()

In [ ]:
var_names = ["floor", "log_u", "floor|county", "floor|county_sigma", "sigma"]
az.plot_trace(
    data=covariate_results, var_names=var_names, compact=True, chain_prop={"ls": "-"}
);

In [ ]:
# get log_u values per county
log_u_sample = df.groupby(["county"])["log_u"].mean().values

# compute the slope posterior samples
log_u_slope = covariate_results.posterior["log_u"].values[..., None] * log_u_sample

# Compute the posterior for the floor coefficient when it is Basement
intercepts = (
    covariate_results.posterior.sel(floor_dim="Basement")["floor"]
    + covariate_results.posterior.sel(floor__expr_dim="Basement")["floor|county"]
).values

y_predicted = (intercepts + log_u_slope).reshape(4000, n_counties).T

# reduce the intercepts posterior samples to the mean per county
mean_intercept = intercepts.mean(axis=2)[..., None] + log_u_slope

In [ ]:
fig, ax = plt.subplots()

y_predicted_bounds = np.quantile(y_predicted, q=[0.03, 0.96], axis=1)

sns.scatterplot(
    x=log_u_sample,
    y=y_predicted.mean(axis=1),
    alpha=0.8,
    color="C0",
    s=50,
    label="Mean county-intercept",
    ax=ax,
)
ax.vlines(
    log_u_sample, y_predicted_bounds[0], y_predicted_bounds[1], color="C1", alpha=0.5
)

az.plot_hdi(
    x=log_u_sample,
    y=mean_intercept,
    color="black",
    fill_kwargs={"alpha": 0.1, "label": "Mean intercept HPD"},
    ax=ax,
)

sns.lineplot(
    x=log_u_sample,
    y=mean_intercept.reshape(4000, n_counties).mean(axis=0),
    color="black",
    alpha=0.6,
    label="Mean intercept",
    ax=ax,
)

ax.legend(loc="upper left")
ax.set(
    title="County Intercepts (Covariance Model)",
    xlabel="County-level log uranium",
    ylabel="Intercept estimate",
);

In [ ]:
# generate posterior predictive samples
pooled_model.predict(pooled_results, kind="response")
partial_pooling_model.predict(partial_pooling_results, kind="response")

# stack chain and draw values
pooled_pps = az.extract(pooled_results, group="posterior_predictive")[
    "log_radon"
].values
partial_pooling_pps = az.extract(partial_pooling_results, group="posterior_predictive")[
    "log_radon"
].values

# Generate predictions per county
pooled_pps_df = pd.DataFrame(data=pooled_pps).assign(county=df["county"])
y_pred_pooled = pooled_pps_df.groupby("county").mean().mean(axis=1)

partial_pooling_pps_df = pd.DataFrame(data=partial_pooling_pps).assign(
    county=df["county"]
)
y_pred_partial_pooling = partial_pooling_pps_df.groupby("county").mean().mean(axis=1)

# observed values
y_sample = df.groupby("county")["log_radon"].mean()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

sns.regplot(x=y_sample, y=y_pred_pooled, label="pooled", color="C0", ax=ax)
sns.regplot(
    x=y_sample, y=y_pred_partial_pooling, label="partial pooling", color="C2", ax=ax
)
ax.axhline(y=df["log_radon"].mean(), color="C0", linestyle="--", label="sample mean")
ax.axline(xy1=(1, 1), slope=1, color="black", linestyle="--", label="diagonal")
ax.axhline(
    y=y_pred_partial_pooling.mean(),
    color="C3",
    linestyle="--",
    label="predicted global mean (partial pooling)",
)
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2)
ax.set(
    title="log(Radon + 0.1) Mean per County - Model Comparison",
    xlabel="observed (sample)",
    ylabel="prediction",
    xlim=(0.2, 2.8),
    ylim=(0.2, 2.8),
);